## Imports

In [48]:
import pandas as pd
import numpy as np
import json
import nltk
import streamlit as st
import spacy
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation

import gensim
pyLDAvis.enable_notebook()

# local library
from preproc import *

In [2]:
with open('speeches.json') as f:
    speeches = json.load(f)

In [3]:
bow = create_bow(speeches)

## Preprocess

In [72]:
TOKEN = tokenize(bow)

In [73]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [74]:
# add term_id back to token
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

In [75]:
NOUNS = TOKEN[TOKEN.pos.str.match(r'^NNS?$')]\
    .groupby(['id','speaker']).term_str\
    .apply(lambda x: ' '.join(x))\
    .to_frame()\
    .rename(columns={'term_str':'speech'})

In [76]:
NOUNS

,,speech
id,speaker,
-8KiohxTJ0Y,biden,wife sister folks things clothes count campaig...
-bHGb17kejM,trump,thank warning warning requests sir updates fra...
0pJnrHKoBHY,trump,music election tomorrow weve state cars gonna ...
17Xl75GwOaM,trump,thank vice president place village confidence ...
1IDMMDn2wPU,trump,hmm hello john sound evening its pennsylvania ...
...,...,...
zIVdxsiu_r4,biden,reminder todays event distancing guideline gui...
zYz-QRYtR9U,trump,secretary carson secretary devos dr larry yarn...
zaaTZkqsaxY,trump,nothing thing closing rating everythin...


In [78]:
stop_words = text.ENGLISH_STOP_WORDS.union(['music','applause'])

In [77]:
biden = NOUNS.query('speaker == "biden"').speech
trump = NOUNS.query('speaker == "trump"').speech
pence = NOUNS.query('speaker == "pence"').speech
harris = NOUNS.query('speaker == "harris"').speech

## Scikit-Learn

In [115]:
tf_vectorizer_biden = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_trump = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_pence = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_harris = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
dtm_tf_biden = tf_vectorizer_biden.fit_transform(biden)
dtm_tf_trump = tf_vectorizer_trump.fit_transform(trump)
dtm_tf_pence = tf_vectorizer_pence.fit_transform(pence)
dtm_tf_harris = tf_vectorizer_harris.fit_transform(harris)

In [116]:
# for TF DTM
lda_tf_biden = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_biden.fit(dtm_tf_biden)
lda_tf_trump = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_trump.fit(dtm_tf_trump)
lda_tf_pence = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_pence.fit(dtm_tf_pence)
lda_tf_harris = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_harris.fit(dtm_tf_harris)

LatentDirichletAllocation(n_components=8, random_state=0)

In [117]:
pyLDAvis.sklearn.prepare(lda_tf_biden, dtm_tf_biden, tf_vectorizer_biden)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.046190 -0.004907       1        1  30.027596
2     -0.062672 -0.044404       2        1  17.996617
3      0.049038 -0.165294       3        1  15.533991
4      0.093360 -0.054026       4        1  11.773192
6      0.001672  0.052574       5        1  10.948844
7      0.182996  0.137894       6        1   4.933514
1     -0.131685 -0.015787       7        1   4.616925
0     -0.178898  0.093950       8        1   4.169322, topic_info=           Term        Freq       Total Category  logprob  loglift
443        jobs  243.000000  243.000000  Default  30.0000  30.0000
323       folks  160.000000  160.000000  Default  29.0000  29.0000
449     justice   75.000000   75.000000  Default  28.0000  28.0000
887       trump  228.000000  228.000000  Default  27.0000  27.0000
138     climate   75.000000   75.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
136       class    6.826351   68.372555   Topic8  -4.9526   0.8732
746      school    6.887316  131.167139   Topic8  -4.9437   0.2306
343  generation    5.874371   40.919534   Topic8  -5.1028   1.2364
860      theyre    6.419972  120.165413   Topic8  -5.0140   0.2480
642       power    6.154871   87.099417   Topic8  -5.0562   0.5276

[534 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2         1  0.022591  access
2         3  0.451824  access
2         4  0.225912  access
2         5  0.112956  access
2         6  0.067774  access
...     ...       ...     ...
963       5  0.165334   youre
963       6  0.035429   youre
966       2  0.357228     zip
966       5  0.119076     zip
966       8  0.476304     zip

[1548 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 4, 5, 7, 8, 2, 1])

In [118]:
pyLDAvis.sklearn.prepare(lda_tf_trump, dtm_tf_trump, tf_vectorizer_trump)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.042939  0.067006       1        1  28.160824
6     -0.048718 -0.068436       2        1  21.243302
7     -0.065189 -0.090505       3        1  19.630934
4      0.076124 -0.083235       4        1  13.599494
5     -0.059420  0.071339       5        1  10.164291
3     -0.092489  0.179209       6        1   3.430295
2      0.259358  0.065640       7        1   2.045818
1     -0.026727 -0.141018       8        1   1.725043, topic_info=         Term        Freq       Total Category  logprob  loglift
111     biden  485.000000  485.000000  Default  30.0000  30.0000
39    america  319.000000  319.000000  Default  29.0000  29.0000
631       joe  411.000000  411.000000  Default  28.0000  28.0000
195     china  407.000000  407.000000  Default  27.0000  27.0000
902    police  248.000000  248.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
885      plan    6.124212  179.359848   Topic8  -5.0428   0.6828
1059  schools    5.453847  124.779625   Topic8  -5.1588   0.9297
195     china    6.124721  407.721756   Topic8  -5.0428  -0.1383
1319     wars    5.265146  106.173626   Topic8  -5.1940   1.0560
1352    woman    5.476633  178.918693   Topic8  -5.1546   0.5735

[528 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         1  0.239768  ability
0         2  0.047954  ability
0         3  0.095907  ability
0         4  0.239768  ability
0         5  0.191814  ability
...     ...       ...      ...
1368      7  0.007787     york
1368      8  0.015575     york
1369      2  0.679612    yorks
1369      3  0.061783    yorks
1369      4  0.185349    yorks

[1758 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 8, 5, 6, 4, 3, 2])

In [119]:
pyLDAvis.sklearn.prepare(lda_tf_harris, dtm_tf_harris, tf_vectorizer_harris)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.010020  0.057976       1        1  55.237853
1     -0.017090 -0.133677       2        1  26.654624
7     -0.188679  0.015946       3        1   7.688720
6      0.034276 -0.102313       4        1   5.746847
3      0.096101  0.001762       5        1   4.454951
4      0.037015  0.059565       6        1   0.173617
5      0.024199  0.050371       7        1   0.021693
0      0.024199  0.050371       8        1   0.021693, topic_info=           Term       Freq      Total Category  logprob  loglift
876        year  33.000000  33.000000  Default  30.0000  30.0000
61        biden  95.000000  95.000000  Default  29.0000  29.0000
415         joe  83.000000  83.000000  Default  28.0000  28.0000
103        care  56.000000  56.000000  Default  27.0000  27.0000
831        vote  60.000000  60.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
332       group   0.001787   4.282791   Topic8  -6.7856   0.6542
790       today   0.001787  50.923715   Topic8  -6.7856  -1.8215
686  securities   0.001787   2.493557   Topic8  -6.7856   1.1951
621    proposal   0.001787   2.493557   Topic8  -6.7856   1.1951
518   murderers   0.001787   2.493557   Topic8  -6.7856   1.1951

[508 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
0         2  0.419043         ability
0         5  0.419043         ability
1         1  0.442567        abortion
1         5  0.442567        abortion
6         1  0.402398  accountability
...     ...       ...             ...
879       1  0.526205            york
879       3  0.420964            york
882       1  0.753584           youre
882       2  0.060287           youre
882       3  0.180860           youre

[658 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 8, 7, 4, 5, 6, 1])

In [120]:
pyLDAvis.sklearn.prepare(lda_tf_pence, dtm_tf_pence, tf_vectorizer_pence)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.103638 -0.036389       1        1  65.450560
3     -0.103464 -0.036303       2        1  19.098129
5     -0.009892  0.068246       3        1   4.426767
0     -0.097511 -0.003963       4        1   4.276821
4      0.187182  0.130506       5        1   2.684838
2      0.144395 -0.204427       6        1   2.296217
6     -0.033021  0.070693       7        1   1.742853
7      0.015947  0.011636       8        1   0.023815, topic_info=        Term        Freq       Total Category  logprob  loglift
213   donald  236.000000  236.000000  Default  30.0000  30.0000
19   america  255.000000  255.000000  Default  29.0000  29.0000
686    space   54.000000   54.000000  Default  28.0000  28.0000
767    trump  288.000000  288.000000  Default  27.0000  27.0000
51     biden  292.000000  292.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
604  records    0.003898    4.215284   Topic8  -6.7286   1.3566
749    thing    0.003898   42.004109   Topic8  -6.7286  -0.9425
406   leader    0.003898   22.845740   Topic8  -6.7286  -0.3335
754   ticket    0.003898   12.548434   Topic8  -6.7286   0.2657
773  uniform    0.003898   33.535452   Topic8  -6.7286  -0.7173

[491 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1         1  0.274669     access
1         2  0.549338     access
4         1  0.532022     action
4         2  0.106404     action
4         3  0.053202     action
...     ...       ...        ...
830       4  0.037186  yesterday
830       6  0.037186  yesterday
830       7  0.074372  yesterday
831       1  0.244379       york
831       2  0.733138       york

[1021 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 6, 1, 5, 3, 7, 8])

In [121]:
biden_viz = pyLDAvis.sklearn.prepare(lda_tf_biden, dtm_tf_biden, tf_vectorizer_biden)
pyLDAvis.save_html(biden_viz, 'biden_viz.html')

In [122]:
trump_viz = pyLDAvis.sklearn.prepare(lda_tf_trump, dtm_tf_trump, tf_vectorizer_trump)
pyLDAvis.save_html(trump_viz, 'trump_viz.html')

In [123]:
harris_viz = pyLDAvis.sklearn.prepare(lda_tf_harris, dtm_tf_harris, tf_vectorizer_harris)
pyLDAvis.save_html(harris_viz, 'harris_viz.html')

In [124]:
pence_viz = pyLDAvis.sklearn.prepare(lda_tf_pence, dtm_tf_pence, tf_vectorizer_pence)
pyLDAvis.save_html(pence_viz, 'pence_viz.html')